In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [2]:
tf.test.is_built_with_cuda()

True

In [3]:
print(tf.version.VERSION)

2.10.1


# Train Code

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import models
from keras import layers
from keras import losses
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # Load the dataset
# # Assumes you have a dataset of hand gesture images stored in 'data' folder with corresponding labels as subfolders
# data_dir = 'Train/'

# # Data preprocessing
# image_size = (380, 250)
# batch_size = 50

# train_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     data_dir,
#     validation_split=0.2,
#     subset="training",
#     seed=123,
#     image_size=image_size,
#     batch_size=batch_size,
#     label_mode='categorical'
# )
image_datagen = ImageDataGenerator(rescale=1./255,
                                  #rotation_range=20,
                                  width_shift_range=0.2,
                                  height_shift_range=0.2,
                                  shear_range=0.2,
                                  zoom_range=0.2,
                                  horizontal_flip=False,
                                  dtype='float32',
                                  validation_split=0.2)

data_dir = 'Train/'
test_data_dir = 'Test/'

train_ds = image_datagen.flow_from_directory(    data_dir,
                                                 target_size = (400, 300),
                                                 batch_size  = 10,
                                                 class_mode  = 'categorical',
                                                 subset='training'
                                                 )

# val_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     data_dir,
#     validation_split=0.2,
#     subset="validation",
#     seed=123,
#     image_size=image_size,
#     batch_size=batch_size,
#     label_mode='categorical'
# )

validation_ds = image_datagen.flow_from_directory(   data_dir,
                                                     target_size = (400, 300),
                                                     batch_size  = 10,
                                                     class_mode  = 'categorical',
                                                     subset='validation'
                                                     )


test_ds = image_datagen.flow_from_directory(     test_data_dir,
                                                 target_size = (400, 300),
                                                 batch_size  = 5,
                                                 class_mode  = 'categorical',
                                                 )

#class_names = train_ds.class_names
#num_classes = 16 #len(class_names)
#print(class_names)

# # Data augmentation
# data_augmentation = keras.Sequential(
#     [
#         layers.experimental.preprocessing.Rescaling(1.0 / 255),
#         #layers.experimental.preprocessing.RandomFlip("horizontal"),
#         #layers.experimental.preprocessing.RandomRotation(0.1),
#         #layers.experimental.preprocessing.RandomZoom(0.1),
#     ]
# )


# Build the CNN model
model = keras.Sequential([
    #data_augmentation,
    layers.Conv2D(256, kernel_size=(3, 3), activation='relu', input_shape=(400, 300, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(64, activation='relu'),
    #layers.Dense(64, activation='relu'),
    #layers.Dense(64, activation='relu'),
    layers.Dense(18, activation='softmax')
])

model.summary()

# Compile the model
model.compile(optimizer='adam',
              #loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              loss = 'categorical_crossentropy',
              #loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
#epochs = 5
#model.fit(train_ds, validation_data=val_ds, epochs=epochs)
#model.fit(train_ds, validation_data = validation_ds, epochs = epochs)

# Save the trained model
#model.save("hand_gesture_model")

# start to train the model
history = model.fit(    train_ds,
                        validation_data  = validation_ds,
                        #validation_split=0.2,
                        epochs           = 5 )

print("Train accuracy : ", str(history.history['accuracy'][-1]))
print("Val accuracy : ", str(history.history['val_accuracy'][-1]))

print("Train loss : ", str(history.history['loss'][-1]))
print("Val loss : ", str(history.history['val_loss'][-1]))


# To show the evaluation resuls of the Test set
test_loss, test_acc = model.evaluate(test_ds)
print(' ')
print('Loss for Testset:', test_loss)
print('Accuracy for Testset:', test_acc)
print(' ')


Found 5490 images belonging to 18 classes.
Found 1361 images belonging to 18 classes.
Found 1914 images belonging to 18 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 398, 298, 256)     7168      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 199, 149, 256)    0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 197, 147, 256)     590080    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 98, 73, 256)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 96, 71, 128)       2950

In [ ]:
# ---------------------------------------------------------------------------------------------------------------------
To plot result figures
import matplotlib.pyplot    as plt

#---------- loss figure -----------------
plt.plot(history.history['loss'])
plt.plot(history.history["val_loss"])
plt.title('train_loss')
plt.ylabel('loss')
plt.xlabel('Epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.show()
#------------------------------------
plt.clf()
#-------------- accuracy figure -------
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
#-----------------------------------


In [1]:
#from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ImageDataGenerator can create an object used to generate batches of image data
image_datagen = ImageDataGenerator(rescale = 1./255, dtype='float32', validation_split=0.2)
data_dir = 'Train/'
# The ImageDataGenerator class has the method flow_from_directory() 
# wihch can read the images from folders containing images.
train_set = image_datagen.flow_from_directory(  data_dir,
                                                 target_size = (960, 540),
                                                 batch_size  = 10,
                                                 class_mode  = 'categorical',
                                                 subset='training'
                                                 )

validation_set = image_datagen.flow_from_directory( data_dir,
                                                 target_size = (960, 540),
                                                 batch_size  = 10,
                                                 class_mode  = 'categorical',
                                                 subset='validation'
                                                 )

test_set = image_datagen.flow_from_directory('   Test/',
                                                 target_size = (960, 540),
                                                 batch_size  = 10,
                                                 class_mode  = 'categorical',
                                                 )

NameError: name 'ImageDataGenerator' is not defined

# Test Code

In [9]:
# Load the saved model
#model = keras.models.load_model("hand_gesture_model")

# Set the directory path for the test images folder
test_data_dir = 'Test/'

# Load and preprocess the test images
test_image_generator = ImageDataGenerator(rescale=1.0 / 255)
test_data = test_image_generator.flow_from_directory(
    test_data_dir,
    #target_size=image_size,
    batch_size=10,
    class_mode='sparse',
    shuffle=False
)

# Make predictions on the test images
predictions = model.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)

# Obtain the true labels for the test images
true_classes = test_data.labels

# Calculate accuracy
accuracy = np.mean(predicted_classes == true_classes)
accuracy_percentage = round(accuracy * 100, 4)  # Round accuracy to 4 decimal places
print("Accuracy:",accuracy_percentage,"%")

# Print the predicted classes for each test image
for i, image_path in enumerate(test_data.filepaths):
    image_name = image_path.split('/')[-1]  # Extract the image file name
    predicted_class = class_names[predicted_classes[i]]
    true_class = class_names[true_classes[i]]
    print("Image:", image_name)
    print("Predicted class:", predicted_class)
    print("True class:", true_class)
    print()

Found 1039 images belonging to 16 classes.


InvalidArgumentError: Graph execution error:

Detected at node 'sequential/flatten/Reshape' defined at (most recent call last):
    File "C:\Users\SHAKIB\anaconda3\envs\new_gpu\lib\runpy.py", line 193, in _run_module_as_main
      "__main__", mod_spec)
    File "C:\Users\SHAKIB\anaconda3\envs\new_gpu\lib\runpy.py", line 85, in _run_code
      exec(code, run_globals)
    File "C:\Users\SHAKIB\anaconda3\envs\new_gpu\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\Users\SHAKIB\anaconda3\envs\new_gpu\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "C:\Users\SHAKIB\anaconda3\envs\new_gpu\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\Users\SHAKIB\anaconda3\envs\new_gpu\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\SHAKIB\anaconda3\envs\new_gp

# Skip from here till below

In [5]:
# Load the saved model
#model = keras.models.load_model("hand_gesture_model")

# Make predictions on new images
image = tf.keras.preprocessing.image.load_img("Test/", target_size=image_size)
input_arr = keras.preprocessing.image.img_to_array(image)
input_arr = np.array([input_arr])  # Convert single image to a batch
predictions = model.predict(input_arr)
predicted_class = class_names[np.argmax(predictions[0])]
print("Predicted class:", predicted_class)


PermissionError: [Errno 13] Permission denied: 'Test/'

In [ ]:
# Load the saved model
#model = keras.models.load_model("hand_gesture_model")

# Set the directory path for the test images folder
test_data_dir = 'Test/'

# Load and preprocess the test images
test_image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)
test_data = test_image_generator.flow_from_directory(
    test_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False
)

# Make predictions on the test images
predictions = model.predict(test_data)
predicted_classes = np.argmax(predictions, axis=1)

# Print the predicted classes for each test image
for i, image_path in enumerate(test_data.filepaths):
    image_name = image_path.split('/')[-1]  # Extract the image file name
    predicted_class = class_names[predicted_classes[i]]
    print("Image:", image_name)
    print("Predicted class:", predicted_class)
    print()
